In [1]:
from pgsql_search.loader import HuggingFaceDatasets

ds = HuggingFaceDatasets("UCSC-VLAA/Recap-COCO-30K")
ds.dataset = ds.dataset.shuffle()
ds = ds.select(list(range(100)))
ds.save_images("../data/images100")
ds = ds.select_columns(["image_filepath", "caption"])
ds.dataset


2024-12-18 17:38:29.687 | INFO     | pgsql_search.loader:__init__:15 - Loading dataset: UCSC-VLAA/Recap-COCO-30K
2024-12-18 17:38:29.695 | INFO     | pgsql_search.loader:save_images:44 - Saving images to folder: ../data/images100


Saving images (num_proc=8):   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['image_filepath', 'caption'],
    num_rows: 100
})

In [2]:
df = ds.dataset.to_pandas()
df

,image_filepath,caption
0,../data/images100/440793.jpg,a couple of skiers on top of a snowy mountain
1,../data/images100/340089.jpg,A group of people who are sitting on couches i...
2,../data/images100/422677.jpg,a boy is riding a skateboard down the street
3,../data/images100/292033.jpg,A bowl of food with steam rising off it on a g...
4,../data/images100/265789.jpg,a person standing on a pair of skis on top of...
...,...,...
95,../data/images100/383293.jpg,The ears of this horse are sticking straight up.
96,../data/images100/546164.jpg,Two zebras standing in the middle of a sandy area
97,../data/images100/17909.jpg,A refrigerator opened with several food items ...
98,../data/images100/399764.jpg,A man is walking with a cow who's sporting a r...


In [3]:
from pgsql_search.database import PostgreSQLDatabase, ColumnType

PostgreSQLDatabase.create_database("my_database")

2024-12-18 17:38:30.191 | INFO     | pgsql_search.database:create_database:296 - Created database 'my_database'


In [4]:
with PostgreSQLDatabase("my_database") as db:
    # First, create the table with just an ID column
    db.initialize_table("image_metadata")
    db.add_column("image_filepath", ColumnType.TEXT, nullable=False)
    db.add_column("caption", ColumnType.TEXT, nullable=True)


    db.insert_dataframe(df)

2024-12-18 17:38:30.270 | INFO     | pgsql_search.database:connect:86 - Connected to database
2024-12-18 17:38:30.283 | INFO     | pgsql_search.database:setup_pgvector_extension:103 - pgvector extension initialized
2024-12-18 17:38:30.287 | INFO     | pgsql_search.database:initialize_table:125 - Initialized table 'image_metadata' with ID column 'id'
2024-12-18 17:38:30.289 | INFO     | pgsql_search.database:add_columns:174 - Added 1 new columns to image_metadata
2024-12-18 17:38:30.290 | INFO     | pgsql_search.database:add_columns:174 - Added 1 new columns to image_metadata
2024-12-18 17:38:30.297 | INFO     | pgsql_search.database:insert_dataframe:226 - Inserted batch 1 (100/100 rows)
2024-12-18 17:38:30.297 | INFO     | pgsql_search.database:insert_dataframe:230 - Successfully inserted 100 rows into image_metadata
2024-12-18 17:38:30.297 | INFO     | pgsql_search.database:disconnect:95 - Disconnected from database
